<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/LagrangePointTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root
import torch
from plotly.subplots import make_subplots
import plotly.graph_objects as go
torch.set_default_dtype(torch.float64)

In [14]:

m1, m2 = (50,30)
r1=torch.tensor([-5.,0,0])
r2=torch.Tensor([5.,0,0])
rc = (m1*r1+m2*r2)/(m1+m2)
w = ((m1+m2)/(torch.linalg.norm(r2-r1))**3)**(0.5) * torch.tensor([0,0,1.])

#set origin relative to center of mass
r1=r1-rc
r2=r2-rc
r1,r2

(tensor([-3.7500,  0.0000,  0.0000]), tensor([6.2500, 0.0000, 0.0000]))

In [17]:
def f(r):
    r=torch.from_numpy(r)
    return -m1/((torch.linalg.norm(r-r1))**3)*(r-r1) - m2/((torch.norm(r-r2))**3)*(r-r2) - torch.cross(w, torch.cross(w,r))

In [18]:
res=root(f,[-10.,0.,0.])
res

    fjac: array([[-1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0., -1.]])
     fun: array([ 3.41948692e-14, -0.00000000e+00,  0.00000000e+00])
 message: 'The solution converged.'
    nfev: 10
     qtf: array([-6.22823648e-09,  0.00000000e+00,  0.00000000e+00])
       r: array([-0.30342957,  0.        ,  0.        ,  0.13179135,  0.        ,
        0.21179135])
  status: 1
 success: True
       x: array([-11.52524009,   0.        ,   0.        ])

In [66]:
def rhs(t,vec):
    # print(torch.split(torch.from_numpy(vec),3))
    v,r =torch.split(torch.from_numpy(vec),3) if isinstance(vec,np.ndarray) else torch.split(vec,3)
    dv=-m1/((torch.linalg.norm(r-r1))**3)*(r-r1) - m2/((torch.linalg.norm(r-r2))**3)*(r-r2) - torch.cross(w, torch.cross(w,r)) - 2*torch.cross(w,v)
    dr = v  
    return torch.concat([dv,dr])

In [97]:
def jac(t,vec):
    vec = torch.from_numpy(vec) if isinstance(vec, np.ndarray) else vec
    return torch.autograd.functional.jacobian(lambda v: rhs(t,v), vec)

In [98]:
# vr0 = torch.tensor([0.0,-0.0, 0.0, 12.38038367, 0.0, 0.])     #L2
# vr0 = torch.tensor([0.0,-0.0, 0.0, 1.77342625, 0.0, 0.])      #L1
# vr0 = torch.tensor([0.0, 0.0, 0.0, 1.25, 8.66025404, 0.])     #L4
# vr0 = torch.tensor([0.0, 0.0, 0.0, 1.25, -8.66025404, 0.])    #L5
# vr0 = torch.tensor([0.0, 0.0, 0.0, -11.52524009, 0., 0.])     #L1
vr0 = torch.tensor([-0., -0., 0, 12.37, 0, 0]) #start from center of mass
tend=50
res=solve_ivp(rhs,(0,tend),vr0,method='Radau',dense_output=True, jac = jac)

In [99]:
def equispaced_t(tend, dmax=0.005):
    tplot=[]
    t=0
    v0,r0 = torch.split(vr0,3)
    tinc=1
    while t<tend:
        while True:
            v,r=torch.split(torch.from_numpy(res.sol(t+tinc)),3)
            dist = torch.linalg.norm(r-r0)
            if dist>dmax:
                tinc=tinc/2
            elif dist<dmax/4:
                tinc=tinc*1.5
            else:
                tplot.append(t+tinc)
                t=t+tinc
                r0=r
                break
    return np.array(tplot)


In [102]:
# tplot = np.linspace(0,tend,1000)
tplot = equispaced_t(tend,0.02)

In [103]:

dx,dy,dz,x,y,z=res.sol(tplot)
fig=make_subplots()
fig.add_trace(go.Scatter(x=x,y=y,mode='lines'))
fig.update_layout(width=800,height=800,template='plotly_dark')

In [ ]:
G=6.67e-11
Mmoon = 7.35e22
Mearth = 5.97e24
Rmoon_earth = 400e6

In [ ]:
omega = (G*(Mmoon+Mearth)/Rmoon_earth**3)**(0.5)

In [ ]:
2*np.pi/omega/3600/24

28.976700546754795